In [50]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet.py
import os, re, glob, json
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile

from sklearn.model_selection import train_test_split

from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator, make_dir

In [56]:
FILE_PATH = '/home/matt/repos/ohia.ai/data'
MODEL_NAME = 'mobilenetv1'
SEED = 0
BATCH_SIZE = 32

ALPHA = None
FILTERED = False
AUGMENTATION = False

IMAGE_DIR = ('filtered_' if FILTERED else '') + 'images_preprocessed'
MODEL_PATH = (
    f'{FILE_PATH}/models/{MODEL_NAME}' +
    f'_seed-{SEED}' +
    f'_batch_size-{BATCH_SIZE}' +
    (f'_{ALPHA:0.2f}' if ALPHA else '') +
    ('_filtered' if FILTERED else '') +
    ('_augmentation' if AUGMENTATION else '')
)

make_dir(MODEL_PATH)

In [3]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]

# encode label names with ids
fle = FastLabelEncoder()
label_ids = fle.fit_transform(full_label_list)

# save id2label map
id2label = {int(fle.transform([label])):label for label in np.unique(full_label_list)}
with open(f'{MODEL_PATH}/plantnet_classes.json', 'w') as fp:
    json.dump(id2label, fp)

In [4]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, label_ids, test_size=0.1, random_state=SEED)

In [5]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(
    train_files, train_ids, n_classes,
    batch_size=BATCH_SIZE,
    augment=AUGMENTATION
)
valid_generator = PlantNetGenerator(
    valid_files, valid_ids, n_classes,
    batch_size=BATCH_SIZE,
    augment=AUGMENTATION,
    shuffle=False
)

In [ ]:
def get_model(model_name):
    
    # load pretrained ImageNet model
    if MODEL_NAME == 'mobilenetv1':
        base_model = MobileNet(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )        
    elif MODEL_NAME == 'mobilenetv2':
        base_model = MobileNetV2(
            input_shape=(224,224,3),
            alpha=ALPHA,
            weights='imagenet',
            include_top=False
        )
    elif MODEL_NAME == 'nasnetmobile':
        base_model = NASNetMobile(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )
    else:
        assert ValueError(
            f'model_name parameter must be one of the following'
            ' "mobilenetv1", '
            ' "mobilenetv2", '
            ' "nasnetmobile"'
        )

    # set freeze all layers
    for layer in base_model.layers:
        layer.trainable = False

    # map ImageNet features to plants
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    # compile the model
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(
        optimizer=Adam(lr=0.0001),
        loss='categorical_crossentropy',
        metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
    )
    return model

In [6]:

# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{MODEL_PATH}/weights.h5',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
    ),
]

# train model 
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=20,
    epochs=100,    
)

Epoch 1/100
592/592 [==============================] - 70s 118ms/step - loss: 2.1411 - top_1_accuracy: 0.4303 - top_3_accuracy: 0.6335 - top_5_accuracy: 0.7298 - val_loss: 1.9452 - val_top_1_accuracy: 0.4510 - val_top_3_accuracy: 0.6813 - val_top_5_accuracy: 0.7812
Epoch 2/100
592/592 [==============================] - 62s 105ms/step - loss: 1.5140 - top_1_accuracy: 0.5748 - top_3_accuracy: 0.7787 - top_5_accuracy: 0.8541 - val_loss: 1.7407 - val_top_1_accuracy: 0.5034 - val_top_3_accuracy: 0.7274 - val_top_5_accuracy: 0.8139
Epoch 3/100
592/592 [==============================] - 64s 108ms/step - loss: 1.1712 - top_1_accuracy: 0.6646 - top_3_accuracy: 0.8488 - top_5_accuracy: 0.9093 - val_loss: 1.6539 - val_top_1_accuracy: 0.5274 - val_top_3_accuracy: 0.7548 - val_top_5_accuracy: 0.8298
Epoch 5/100
592/592 [==============================] - 64s 107ms/step - loss: 1.0793 - top_1_accuracy: 0.6884 - top_3_accuracy: 0.8668 - top_5_accuracy: 0.9220 - val_loss: 1.6562 - val_top_1_accuracy: 0

592/592 [==============================] - 64s 109ms/step - loss: 0.4307 - top_1_accuracy: 0.8897 - top_3_accuracy: 0.9736 - top_5_accuracy: 0.9898 - val_loss: 1.5124 - val_top_1_accuracy: 0.5635 - val_top_3_accuracy: 0.7937 - val_top_5_accuracy: 0.8663
Epoch 30/100
592/592 [==============================] - 65s 109ms/step - loss: 0.4308 - top_1_accuracy: 0.8907 - top_3_accuracy: 0.9754 - top_5_accuracy: 0.9899 - val_loss: 1.5294 - val_top_1_accuracy: 0.5620 - val_top_3_accuracy: 0.7923 - val_top_5_accuracy: 0.8644

Epoch 00030: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
Epoch 31/100
592/592 [==============================] - 64s 108ms/step - loss: 0.4277 - top_1_accuracy: 0.8938 - top_3_accuracy: 0.9739 - top_5_accuracy: 0.9899 - val_loss: 1.5230 - val_top_1_accuracy: 0.5630 - val_top_3_accuracy: 0.7889 - val_top_5_accuracy: 0.8654
Epoch 00031: early stopping


In [58]:
# save results
results = pd.DataFrame(model.history.history)
results.to_csv(f'{MODEL_PATH}/results.csv', index=False)

# print best results
best_results = results.iloc[results.val_top_3_accuracy.values.argmax()]
print(best_results)

val_loss              1.500317
val_top_1_accuracy    0.564904
val_top_3_accuracy    0.796635
val_top_5_accuracy    0.867788
loss                  0.498099
top_1_accuracy        0.868032
top_3_accuracy        0.964791
top_5_accuracy        0.984428
lr                    0.000025
Name: 20, dtype: float64


In [8]:
# save best model
if save_model:
    model = get_model(model_name)
    best_weights = glob.glob(f'{FILE_PATH}/weights/{MODEL_NAME}**')
    best_weights = np.sort(best_weights)[-1]
    model.load_weights(best_weights)
    tfjs.converters.save_keras_model(model, f'{FILE_PATH}/models/{MODEL_NAME}')

In [ ]:
# n_units: 1024,  val_top_1_accuracy: 0.6450,  val_top_3_accuracy: 0.8559,  val_top_5_accuracy: 0.9219